In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [7986, 17783, 3370, 16357, 950, 19108, 9573, 470, 8595, 3850, 12858, 8024, 13473, 19186, 3968, 3426, 17594, 11532, 17386, 2755, 3700, 421, 11220, 16931, 13677, 15984, 12648, 18805, 1151, 10925, 2402, 6960, 1730, 2620, 214, 11345, 11547, 2918, 1960, 14143, 1658, 10568, 4064, 20158, 9802, 5042, 12634, 7936, 1296, 19253, 13132, 11456, 9040, 10961, 10801, 6632, 18747, 10142, 19629, 17643, 7503, 19529, 3461, 10797, 7309, 2887, 1149, 17934, 11003, 68, 12236, 14969, 7104, 12239, 4475, 14409, 11390, 19234, 7786, 12603, 18083, 4270, 19004, 3735, 17541, 14772, 18801, 215, 19427, 19581, 17071, 11061, 4572, 19525, 10021, 12681, 11874, 13039, 17924, 17325, 1350, 5718, 5302, 3288, 13819, 14625, 3003, 384, 10483, 15201, 9376, 9279, 15542, 14313, 15986, 10885, 2995, 12019, 7212, 14025, 2741, 15651, 1051, 13284, 19337, 15417, 16664, 12056, 10032, 16518, 18499, 7549, 11948, 12458, 4649, 15262, 14642, 10331, 8436, 15291, 1002, 13320, 9789, 4553, 10

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'pret')
# profiler.disable()

Epoch [1/500], Loss: 1.3961, Accuracy: 0.2552, Edge Accuracy: [0.9999960749060409, 0.9996950039427049, 0.9126468692677606]
Class 0: AUC-ROC- 0.5407, F1 Score- 0.0000; Class 1: AUC-ROC- 0.4160, F1 Score- 0.1127; Class 2: AUC-ROC- 0.3797, F1 Score- 0.0377; Class 3: AUC-ROC- 0.5727, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4773; Macro-Average F1 Score: 0.0376
Validation Loss: 1.3851, Validation Accuracy: 0.2250, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.5956, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3348, F1 Score- 0.0000; Class 2: AUC-ROC- 0.4661, F1 Score- 0.0000; Class 3: AUC-ROC- 0.8199, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5541; Macro-Average F1 Score: 0.0000
Test Loss: 1.3856, Test Accuracy: 0.2450, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.6178, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3011, F1 Score- 0.0187; Class 2: AUC-ROC- 0.4612, F1 Score- 0.0000; Class 3: AUC-ROC- 0.7751, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5388; Macro-Average F1 Score: 0.0047

Epoch [11/500]

([0.245,
  0.28,
  0.2975,
  0.3075,
  0.3275,
  0.375,
  0.42,
  0.5125,
  0.585,
  0.6925,
  0.7775,
  0.8225,
  0.8675,
  0.8875,
  0.915,
  0.935,
  0.945,
  0.945,
  0.9425,
  0.945],
 [0.5388,
  0.6554,
  0.75535,
  0.8302624999999999,
  0.8832583333333334,
  0.919175,
  0.9441999999999999,
  0.9601000000000002,
  0.9703999999999999,
  0.9778333333333333,
  0.9830583333333334,
  0.9868916666666667,
  0.9893916666666668,
  0.9913000000000001,
  0.992675,
  0.9936916666666666,
  0.9945333333333334,
  0.9953333333333333,
  0.9957250000000001,
  0.9960833333333333],
 [0.004672897196261682,
  0.0,
  0.0,
  0.0,
  0.0,
  0.023809523809523808,
  0.045454545454545456,
  0.08473389355742297,
  0.13848322729901677,
  0.182146972218666,
  0.2719937405252091,
  0.3475014582928252,
  0.38307416267942584,
  0.41324072866397116,
  0.46591717315401526,
  0.5614617659959736,
  0.6345622797614647,
  0.6549382646680236,
  0.669141751209814,
  0.672100745813542])

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'pret')

Test Loss: 1.1211, Test Accuracy: 0.9550, Test Edge Accuracy: []
Class 0: AUC-ROC- 1.0000, F1 Score- 1.0000; Class 1: AUC-ROC- 0.9991, F1 Score- 0.9751; Class 2: AUC-ROC- 0.9948, F1 Score- 0.8772; Class 3: AUC-ROC- 0.9982, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.9980; Macro-Average F1 Score: 0.7131


(0.955, 0.9980166666666667, 0.7130793401413983)